In [ ]:
import numpy as np
from mat4py import loadmat
import scipy.io
from copy import deepcopy
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.model_selection import StratifiedKFold

import tensorflow 
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

import matplotlib.pyplot as plot
from sklearn.metrics import confusion_matrix
import csv
from tabnanny import verbose

In [ ]:
def My_Model(X,y,type_model,f=100,s=70,t=50,l=6):
    try:
        # Selection of the model
        if(type_model == 'MLP'):
            ''' Multi Layer Perceptron '''
            model = MLPClassifier(hidden_layer_sizes=(f,s,t,l),
                            max_iter=1000,
                            activation = 'tanh',
                            solver='adam', 
                            #warm_start=True,
                            shuffle=True,
                            early_stopping=False)
            
        elif(type_model == 'SVML'):
            ''' Support Vector Machine '''
            model = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo')
        elif(type_model == 'SVMR'):
            model = svm.SVC(kernel='rbf', C=1, decision_function_shape='ovo')
            
        elif(type_model == 'LDA'):
            ''' Linear Discriminant Analysis '''
            model = LinearDiscriminantAnalysis()
            
        elif(type_model == 'NN'):
            ''' Neural Network '''
            input_shape = (X.shape[1],)
            kfold = StratifiedKFold(n_splits=5, shuffle=True) # 5-fold
            loss_per_fold = []
            accuracy_per_fold = []
            n_classes = len(np.unique(y))
            trials = len(y)/6
            m1 = 0
            m2 = 0
            m3 = 0
            m4 = 0
            m5 = 0
            m6 = 0
            mov_acc = []
            mov_acc_per_fold = []
            y_pred_arr = []
            y_true = []
            
            y_categorical = to_categorical(y-1, n_classes)
            
            for train_index, test_index in kfold.split(X, y):
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y_categorical[train_index], y_categorical[test_index] 
                len_per_fold = len(y_test)
                
                # NN Arquitecture
                model = Sequential()
                model.add(keras.layers.InputLayer(input_shape=input_shape))
                model.add(Dense(f, activation='relu'))
                model.add(Dense(s, activation='relu'))
                model.add(Dense(t, activation='relu'))
                model.add(Dense(l, activation='softmax'))
                
                # Compile model and train
                model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
                history = model.fit(X_train, y_train, epochs=120, batch_size=30, verbose=0)
                
                ##########################################################
                # Get classes predictions
                for i in range(0,len(y_test)):
                    y_hat = model.predict(X_test[i,:].reshape(1,-1),verbose=0)
                    y_hat = np.argmax(y_hat, axis=1) + 1
                    y_target = np.argmax(y_test[i], axis=0) + 1
                    
                    y_pred_arr.append(y_hat)
                    y_true.append(y_target)
                    
                    if(y_hat == 1 and y_target == 1):
                        m1 += 1
                    elif(y_hat == 2 and y_target == 2):
                        m2 += 1
                    elif(y_hat == 3 and y_target == 3):
                        m3 += 1
                    elif(y_hat == 4 and y_target == 4):
                        m4 += 1
                    elif(y_hat == 5 and y_target == 5):
                        m5 += 1
                    elif(y_hat == 6 and y_target == 6):
                        m6 += 1   
                ##########################################################
                
                # Metrics
                scores = model.evaluate(X_test, y_test,verbose=0)
                loss_per_fold.append(scores[0])
                accuracy_per_fold.append(scores[1])
                # Obtain accuracy per movement per fold
                mov_acc_per_fold.append(m1/len_per_fold*100),mov_acc_per_fold.append(m2/len_per_fold*100),
                mov_acc_per_fold.append(m3/len_per_fold*100),mov_acc_per_fold.append(m4/len_per_fold*100),
                mov_acc_per_fold.append(m5/len_per_fold*100),mov_acc_per_fold.append(m6/len_per_fold*100)
                
            # Obtain accuracy per movement
            mov_acc.append(np.round(m1/trials*100,3)), mov_acc.append(np.round(m2/trials*100,3)), 
            mov_acc.append(np.round(m3/trials*100,3)), mov_acc.append(np.round(m4/trials*100,3)), 
            mov_acc.append(np.round(m5/trials*100,3)), mov_acc.append(np.round(m6/trials*100,3))
            print("Classes Accuracy:", mov_acc)
            print('NN Average Accuracy: %.3f (%.3f)' % (np.mean(accuracy_per_fold), np.std(accuracy_per_fold)))
            return model, np.mean(accuracy_per_fold), np.std(accuracy_per_fold), np.min(accuracy_per_fold), np.max(accuracy_per_fold), mov_acc, y_pred_arr, y_true, mov_acc_per_fold
        
        ######################################################################
        if(type_model != 'NN'):
            print('Starting K-Fold Cross Validation...')
            # Create StratifiedKFold object.
            k = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
            lst_accu_stratified = []
            model_kfold_cv = model
            trials = len(y)/6
            m1 = 0
            m2 = 0
            m3 = 0
            m4 = 0
            m5 = 0
            m6 = 0
            mov_acc = []
            mov_acc_per_fold = []
            y_pred_arr = []
            y_true = []
                
            for train_index, test_index in k.split(X, y):
                x_train_fold, x_test_fold = X[train_index], X[test_index]
                y_train_fold, y_test_fold = y[train_index], y[test_index]
                model_kfold_cv.fit(x_train_fold, y_train_fold)
                len_per_fold = len(y_test_fold)
                ##########################################################
                # Get classes predictions
                for i in range(0,len(y_test_fold)):
                    y_hat = model_kfold_cv.predict(x_test_fold[i].reshape(1,-1))
                    y_pred_arr.append(y_hat)
                    y_true.append(y_test_fold[i])
                    if(y_hat == y_test_fold[i] and y_hat == 1):
                        m1 += 1
                    elif(y_hat == y_test_fold[i] and y_hat == 2):
                        m2 += 1
                    elif(y_hat == y_test_fold[i] and y_hat == 3):
                        m3 += 1
                    elif(y_hat == y_test_fold[i] and y_hat == 4):
                        m4 += 1
                    elif(y_hat == y_test_fold[i] and y_hat == 5):
                        m5 += 1
                    elif(y_hat == y_test_fold[i] and y_hat == 6):
                        m6 += 1       
                ##########################################################
                lst_accu_stratified.append(model_kfold_cv.score(x_test_fold, y_test_fold))
                #######################################################################
                # Obtain accuracy per movement per fold
                mov_acc_per_fold.append(m1/len_per_fold*100),mov_acc_per_fold.append(m2/len_per_fold*100),
                mov_acc_per_fold.append(m3/len_per_fold*100),mov_acc_per_fold.append(m4/len_per_fold*100),
                mov_acc_per_fold.append(m5/len_per_fold*100),mov_acc_per_fold.append(m6/len_per_fold*100)
                
            # Obtain accuracy per movement
            mov_acc.append(np.round(m1/trials*100,3)), mov_acc.append(np.round(m2/trials*100,3)), 
            mov_acc.append(np.round(m3/trials*100,3)), mov_acc.append(np.round(m4/trials*100,3)), 
            mov_acc.append(np.round(m5/trials*100,3)), mov_acc.append(np.round(m6/trials*100,3))
            print("Classes Accuracy:", mov_acc)
            print(type_model, 'Mean Accuracy: %.3f (%.3f)' % (np.mean(lst_accu_stratified), np.std(lst_accu_stratified)))
        
        # Train Model
        model.fit(X,y)
        return model, np.mean(lst_accu_stratified), np.std(lst_accu_stratified), np.min(lst_accu_stratified), np.max(lst_accu_stratified), mov_acc, y_pred_arr, y_true, mov_acc_per_fold
    except:
        print(type_model, 'Not Found')
        
def My_model_predict(model, X_test,y_test,type_model,pca,scaling,prt=1):
    predictions = []
    # Make Predictions
    if(prt ==1):
        print("Starting Predictions...")
    test_acc = 0
    for i in range(0,len(y_test)):
        X = X_test[i,:]
        #print(X.shape)
        X= np.reshape(X, (1, -1))
        #print(X.shape)
        if X.shape[1] == 40:
            X = pca.transform(X)
            X = scaling.transform(X)
        #print(X.shape)
        
        if type_model != 'NN':
            y_hat = model.predict(X)
            predictions.append(y_hat)
            if y_hat == y_test[i]:
                test_acc += 1
        else:
            y_hat = model.predict(X,verbose=0)
            y_hat = np.argmax(y_hat, axis=1) + 1
            predictions.append(y_hat[0])
            if y_hat == y_test[i]:
                 test_acc += 1
                
        
    score = test_acc/len(y_test)*100
    if(prt == 1):
        print('Test Accuracy: %.3f' % score)
    return score, predictions

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plot.cm.Blues):
   
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        
    cm = confusion_matrix(y_true, y_pred)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
   
    #print(cm)

    figure, axis = plot.subplots()
    im = axis.imshow(cm, interpolation='nearest', cmap=cmap)
    axis.figure.colorbar(im, ax=axis)

    axis.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),

           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')


    plot.setp(axis.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            axis.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="cyan" if cm[i, j] > thresh else "red")
    figure.tight_layout()
    return axis


def PFvsWS(IL,model, X_test,y_test,type_model,pca,scaling,cm_flag=0):
    if(type_model != 'MLP'):
        print("Cannot PF or WS Fit Model")
        return
    
    if(IL == 'PF'):
        print('------- PARTIAL FIT MODEL -------')
    else:
        print('------- WARM START MODEL -------')
        model.warm_start = True
        first_layer = model.hidden_layer_sizes[0]
        second_layer = model.hidden_layer_sizes[1]
        third_layer = model.hidden_layer_sizes[2]
        fourth_layer = model.hidden_layer_sizes[3]
        model.hidden_layer_sizes = (first_layer,second_layer+30,third_layer+50,fourth_layer)
        
    X_new = []
    for i in range(0,len(y_test)):
        X = X_test[i,:]
        X = np.reshape(X, (1, -1))
        X = pca.transform(X)
        X = scaling.transform(X)
        X_new.append(X)
        
    X_new = np.array(X_new)  
    X_new = np.reshape(X_new, (240,36))
    y_new = y_test
    
    percent = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    test_accuracies = []

    for i in range(len(percent)):
        # Copy model to test
        copy_model = deepcopy(model)
        
        x_train = []
        x_test = []
        y_train = []
        y_test = []
        samples = len(X_new)*percent[i]
        idx = int(samples/6)
        
        #print(idx)
        for j in range(len(np.unique(y_new))):
            x_train.append(X_new[j*40:j*40+idx])
            x_test.append(X_new[j*40+idx:j*40+40])
            y_train.append(y_new[j*40:j*40+idx])  
            y_test.append(y_new[j*40+idx:j*40+40])
              
        x_train = np.array(x_train)
        x_test = np.array(x_test)
        y_train = np.array(y_train)
        y_test = np.array(y_test)
        # reshape y to nx1
        y_train = np.reshape(y_train, (len(y_train)*len(y_train[0]),1))
        y_test = np.reshape(y_test, (len(y_test)*len(y_test[0]),1))
        # reshape x to nx36
        x_train = np.reshape(x_train, (len(x_train)*len(x_train[0]),36))
        x_test = np.reshape(x_test, (len(x_test)*len(x_test[0]),36))
        
        y_train = y_train.ravel()
        
        if(IL == 'PF'):
            copy_model.partial_fit(x_train, y_train)
        elif(IL == 'WS'):
            copy_model.fit(x_train, y_train)
        else:
            print('IL not found')
            exit()
        
        mean_acc = []
        
        if(cm_flag == 0):
            for m in range(0,5):
                score, y_pred = My_model_predict(copy_model,x_test,y_test,type_model,pca,scaling,0)
                mean_acc.append(score)
            test_accuracies.append(np.mean(mean_acc))
        else:
            score, y_pred = My_model_predict(copy_model,x_test,y_test,type_model,pca,scaling,1)
            test_accuracies.append(score)
        
            y_pred = np.array(y_pred)
            y_pred = np.reshape(y_pred, (y_pred.shape[0],))
            plot_confusion_matrix(y_test, 
                                y_pred, 
                                classes=np.array(['1','2','3','4','5','6']), 
                                normalize=True, 
                                title=str(percent[i]*100)+'% of Data')
            plot.show()
        
    return np.array(percent), np.array(test_accuracies)

--------------------SINGLE SUBJECT INDEPENDENT--------------------

In [ ]:
################### SSI ############################
ssi_flag = 1
write_csv = 1
if (ssi_flag == 1):
    #classifiers = ['LDA','SVML','SVMR']
    #classifiers = ['MLP1','MLP2','MLP3']
    classifiers = ['NN3']
    for j in range(len(classifiers)):
        clf = classifiers[j]
        All = []
        movs_per = []
        for i in range(0,30):
            if i < 10:
                s = 'S0' + str(i)
            else:   
                s = 'S'+str(i)
            train_path = r'SxxAll'
            test_path = r'Sxx'
            X_train_file = train_path + '\\' + s + 'x_train.mat'
            y_train_file = train_path + '\\' + s + 'y_train.mat'
            X_test_file = test_path + '\\' + s + 'X_1.mat'
            y_test_file = test_path + '\\' + s + 'y.mat'

            ''' Select a Model '''
            name_clf = clf
            # For MLP or NN
            f = 1
            s = 1
            t = 1
            l = 1

            if(name_clf == 'MLP1' or name_clf == 'NN1'):
                f=100
                s=70
                t=30
                l=6
                if(name_clf == 'MLP1'):
                    type_model = 'MLP'
                else:
                    type_model = 'NN'
            elif(name_clf == 'MLP2' or name_clf == 'NN2'):
                f=100
                s=100
                t=100
                l=6
                if(name_clf == 'MLP2'):
                    type_model = 'MLP'
                else:
                    type_model = 'NN'
            elif(name_clf == 'MLP3' or name_clf == 'NN3'):
                f=150
                s=130
                t=130
                l=6
                if(name_clf == 'MLP3'):
                    type_model = 'MLP'
                else:
                    type_model = 'NN'
            else:
                type_model = name_clf
                
            X = X_test_file
            y = y_test_file

            ''' Read Training Subjects Data and Labels'''
            X_subject = scipy.io.loadmat(X)
            y_subject = scipy.io.loadmat(y)
            X_subject = np.array(X_subject['X_1'])
            y_subject = np.array(y_subject['y'])
            X = X_subject
            y = np.reshape(y_subject,(y_subject.shape[0],))

            ''' Feature Selection with PCA '''
            pca = PCA(n_components=36)
            pca.fit(X)
            X = pca.transform(X)
            ''' Scaling '''
            scaling = StandardScaler(with_mean=True, with_std=True)
            scaling.fit(X)
            X = scaling.transform(X)
            ''' Train Model '''
            print("------- SINGLE SUBJECT INDEPENDENT -------")
            print('Subject: ',i)
            print('Model:',type_model)
            model, mean, std, min, max, mov_acc, y_pred, y_true, _ = My_Model(X=X, y=y, type_model=type_model,f=f,s=s,t=t,l=l)
            
            path = r'SSI_CM\\' + name_clf + '\\' + name_clf + '_CM.csv'
            with open(path, 'a', newline='') as file:
                writer = csv.writer(file)
                for idx in range(len(y_true)):
                    writer.writerow([y_true[idx],y_pred[idx][0]])
            
            #### TABLE 1 ####
            result = []
            mean = float(np.round(mean*100,3))
            std = float(np.round(std*100,3))
            min = float(np.round(min*100,3))
            max = float(np.round(max*100,3))
            
            result.append(mean), result.append(std), result.append(min), result.append(max)
            print(result)
            
            if(write_csv == 1):
                with open(r'SSI\\'+name_clf+'_SSI_results.csv','a', newline='') as ssi_results:
                    writer = csv.writer(ssi_results)
                    writer.writerow(result)
                

                #### TABLE 3 ####
                movs_per.append(mov_acc)

                with open(r'SSI\\'+name_clf+'_SSI_Class_results.csv','a', newline='') as class_results:
                    writer = csv.writer(class_results)
                    writer.writerow(mov_acc)
                    
            if i < 10:
                s = 'S0' + str(i)
            else:   
                s = 'S'+str(i)
            title = name_clf + ' ' + s + ' Confusion Matrix'
            plot_confusion_matrix(y_true, 
                                y_pred, 
                                classes=np.array(['1','2','3','4','5','6']), 
                                normalize=True, 
                                title=title)
            path = r'SSI_CM\\'
            path = path + name_clf  + '\\' + name_clf 
            path = path + '_' + s + '_Confusion_Matrix.png'
            plot.savefig(path)
            plot.close()

In [ ]:
############################ CREATE BIG CONFUSION MATRIX ############################
create_big_csv = 0
create_big_confusion_matrix = 1
if(create_big_csv == 0):
    files = ['LDA_CM.csv','SVML_CM.csv','SVMR_CM.csv',
             'MLP1_CM.csv','MLP2_CM.csv','MLP3_CM.csv',
             'NN1_CM.csv','NN2_CM.csv','NN3_CM.csv']
    names = ['LDA','SVML','SVMR','MLP1','MLP2','MLP3','NN1','NN2','NN3']

    for i in range(len(files)):
        file = r'SSI_CM\\' + names[i] + '\\' +files[i]  
        with open(file, 'r') as f:
            df = pd.read_csv(f)
            y_true = np.array(df.iloc[:,0])
            y_pred = np.array(df.iloc[:,1]) 
            
        with open(r'SSI_CM\\SSI_CM.csv','a', newline='') as ssi_cm:
            writer = csv.writer(ssi_cm)
            for idx in range(len(y_true)):
                writer.writerow([y_true[idx],y_pred[idx]])
            
        title = names[i] + ' SSI Confusion Matrix'
        plot_confusion_matrix(y_true, 
                            y_pred, 
                            classes=np.array(['1','2','3','4','5','6']), 
                            normalize=True, 
                            title=title)
        path = r'SSI_CM\\'
        path = path + names[i] + '_Confusion_Matrix.png'
        plot.savefig(path)
        plot.close()
        
if(create_big_confusion_matrix == 1):
    files = 'SSI_CM.csv'
    path = 'SSI_CM\\' + files
    with open(path,'r') as f:
        df = pd.read_csv(f)
        y_true = np.array(df.iloc[:,0])
        y_pred = np.array(df.iloc[:,1]) 
        
    title = ' SSI Confusion Matrix Normalized'
    plot_confusion_matrix(y_true, 
                        y_pred, 
                        classes=np.array(['1','2','3','4','5','6']), 
                        normalize=True, 
                        title=title)
    path = r'SSI_CM\\'
    path = path + 'Confusion_Matrix_Normalized.png'
    plot.savefig(path)
    plot.close()
    
    files = 'SSI_CM.csv'
    path = 'SSI_CM\\' + files
    with open(path,'r') as f:
        df = pd.read_csv(f)
        y_true = np.array(df.iloc[:,0])
        y_pred = np.array(df.iloc[:,1]) 
        
    title = ' SSI Confusion Matrix'
    plot_confusion_matrix(y_true, 
                        y_pred, 
                        classes=np.array(['1','2','3','4','5','6']), 
                        normalize=False, 
                        title=title)
    path = r'SSI_CM\\'
    path = path + 'Confusion_Matrix.png'
    plot.savefig(path)
    plot.close()

In [ ]:
############################ GET RESUME TABLE ############################
get_mean_and_std = 0
if(get_mean_and_std == 1):
    files = ['LDA_SSI_results.csv','SVML_SSI_results.csv','SVMR_SSI_results.csv'
            ,'MLP1_SSI_results.csv','MLP2_SSI_results.csv','MLP3_SSI_results.csv'
            ,'NN1_SSI_results.csv','NN2_SSI_results.csv','NN3_SSI_results.csv'
            ]
    names = ['LDA','SVML','SVMR'
            ,'MLP1','MLP2','MLP3'
            ,'NNET1','NNET2','NNET3'
            ]

    for i in range(len(files)):
        df = pd.read_csv(r'SSI\\'+files[i])
        mean = np.round(np.array(np.mean(df, axis=0)),3)
        with open(r'SSI\\'+files[i], 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(mean)
        with open(r'SSI_All_Table\\Table_2.csv','a',newline='') as f:
            writer = csv.writer(f)
            writer.writerow(mean)
            
    files = ['LDA_SSI_Class_results.csv','SVML_SSI_Class_results.csv','SVMR_SSI_Class_results.csv'
            ,'MLP1_SSI_Class_results.csv','MLP2_SSI_Class_results.csv','MLP3_SSI_Class_results.csv'
            ,'NN1_SSI_Class_results.csv','NN2_SSI_Class_results.csv','NN3_SSI_Class_results.csv'
            ]
    names = ['LDA','SVML','SVMR'
            ,'MLP1','MLP2','MLP3'
            ,'NNET1','NNET2','NNET3'
            ]

    for i in range(len(files)):
        df = pd.read_csv(r'SSI\\'+files[i])
        mean = np.round(np.array(np.mean(df, axis=0)),3)
        std = np.round(np.array(np.std(df, axis=0)),3)
        with open(r'SSI\\'+files[i], 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(mean)
            writer.writerow(std)
        with open(r'SSI_All_class_Table\\Table_Mean_4.csv','a',newline='') as f:
            writer = csv.writer(f)
            writer.writerow(mean)
        with open(r'SSI_All_class_Table\\Table_Std_4.csv','a',newline='') as f:
            writer = csv.writer(f)
            writer.writerow(std)

In [ ]:
file = r'SSI_All_Table\\Table_2.csv'
names = ['LDA','SVML','SVMR','MLP1','MLP2','MLP3','NNET1','NNET2','NNET3']

df = pd.read_csv(file)
mean = np.array(df.iloc[:,0])
std = np.array(df.iloc[:,1])

fig, ax = plot.subplots()
ax.bar(names,mean,
       yerr=std,
       align='center',
       alpha=0.5,
       ecolor='black',
       capsize=10)
plot.ylim(80,100)
plot.xlabel('Classifier')
plot.ylabel('Validation Accuracy')
plot.title('SSI Validation Accuracy')
plot.savefig(r'SSI_All_Table\\Table_2.png')

--------------------LEAVE ONE OUT (LOOP)--------------------

In [ ]:
################### LOOP ALL ############################
loop_flag = 1
write_csv = 1
printer = 0
if(loop_flag == 1):
    classifiers = ['NN2']
    for j in range(len(classifiers)):
        clf = classifiers[j]
        All = []
        movs_per = []
        for i in range(23,30):
            if i < 10:
                s = 'S0' + str(i)
            else:   
                s = 'S'+str(i)
            train_path = r'SxxAll'
            test_path = r'Sxx'
            X_train_file = train_path + '\\' + s + 'x_train.mat'
            y_train_file = train_path + '\\' + s + 'y_train.mat'
            X_test_file = test_path + '\\' + s + 'X_1.mat'
            y_test_file = test_path + '\\' + s + 'y.mat'
            
            ''' Select a Model '''
            name_clf = clf
            # For MLP or NN
            f = 1
            s = 1
            t = 1
            l = 1

            if(name_clf == 'MLP1' or name_clf == 'NN1'):
                f=100
                s=70
                t=30
                l=6
                if(name_clf == 'MLP1'):
                    type_model = 'MLP'
                else:
                    type_model = 'NN'
            elif(name_clf == 'MLP2' or name_clf == 'NN2'):
                f=100
                s=100
                t=100
                l=6
                if(name_clf == 'MLP2'):
                    type_model = 'MLP'
                else:
                    type_model = 'NN'
            elif(name_clf == 'MLP3' or name_clf == 'NN3'):
                f=150
                s=130
                t=130
                l=6
                if(name_clf == 'MLP3'):
                    type_model = 'MLP'
                else:
                    type_model = 'NN'
            else:
                type_model = name_clf

            ''' Read Training Subjects Data and Labels'''
            X_subject = scipy.io.loadmat(X_train_file)
            y_subject = scipy.io.loadmat(y_train_file)
            X_subject = np.array(X_subject['x_train'])
            y_subject = np.array(y_subject['y_train'])
            X = X_subject
            y = np.reshape(y_subject,(y_subject.shape[0],))

            ''' Feature Selection with PCA '''
            pca = PCA(n_components=36)
            pca.fit(X)
            X = pca.transform(X)
            ''' Scaling '''
            scaling = StandardScaler(with_mean=True, with_std=True)
            scaling.fit(X)
            X = scaling.transform(X)
            ''' Train Model '''
            print("------- ALL VS ONE (LOOP) -------")
            print('Subject: ', i)
            print('Model:',type_model)
            model, mean, std, min, max, mov_acc, y_pred, y_true, _  = My_Model(X=X, y=y, type_model=type_model,f=f,s=s,t=t,l=l)
            
            path = r'LOOP_CM\\' + name_clf + '\\' + name_clf + '_CM.csv'
            with open(path, 'a', newline='') as file:
                writer = csv.writer(file)
                for idx in range(len(y_true)):
                    writer.writerow([y_true[idx],y_pred[idx][0]])
                    
            ''' Get Confusion Matrix '''
            if i < 10:
                s = 'S0' + str(i)
            else:   
                s = 'S'+str(i)
            title = name_clf + ' ' + s + ' vs All Confusion Matrix'
            plot_confusion_matrix(y_true, 
                                y_pred, 
                                classes=np.array(['1','2','3','4','5','6']), 
                                normalize=True, 
                                title=title)
            path = r'LOOP_CM\\'
            path = path + name_clf  + '\\' + name_clf 
            path = path + '_' + s + '_Confusion_Matrix.png'
            plot.savefig(path)
            plot.close()
            
            # Read Test Subjects Data
            X_test = scipy.io.loadmat(X_test_file)
            y_test = scipy.io.loadmat(y_test_file)
            X_test = np.array(X_test['X_1'])
            y_test = np.array(y_test['y'])
            y_test = np.reshape(y_test, (y_test.shape[0],))

            ''' Predict (LOOP) '''
            score, y_pred = My_model_predict(model, X_test, y_test, type_model, pca, scaling)
            
            
            ''' Write into CSV'''
            result = []
            mean = float(np.round(mean*100,3))
            std = float(np.round(std*100,3))
            min = float(np.round(min*100,3))
            max = float(np.round(max*100,3))
            score = float(np.round(score,3))
            
            result.append(mean), result.append(std), result.append(min), result.append(max), result.append(score)
            
            if(write_csv == 1):
                with open(r'LOOP\\'+name_clf+'_LOOP_results.csv','a', newline='') as loop_results:
                    writer = csv.writer(loop_results)
                    writer.writerow(result)


                with open(r'LOOP\\'+name_clf+'_LOOP_Class_results.csv','a', newline='') as class_results:
                    writer = csv.writer(class_results)
                    writer.writerow(mov_acc)
            
            if(type_model == 'MLP'):
                ws_path = 'WS\\'
                ws_path = ws_path + name_clf + '_WarmStart.csv'
                pft_path = 'PFT\\'
                pft_path = pft_path + name_clf + '_PartialFit.csv'
                
                ################### PARTIAL FIT TESTING ############################
                get_cm_one_run = 0
                pf_model = deepcopy(model)
                percent, pft_acc = PFvsWS('PF',pf_model, X_test, y_test, type_model, pca, scaling, get_cm_one_run)
                score = np.array(score)
                pft_acc = np.insert(pft_acc,0,score, axis=0)
                pft_acc = np.array(pft_acc)
                pft_acc = np.round(pft_acc,3)
                percent = np.insert(percent, 0, 0.0)
                # Plot percent vs accuracy
                if(printer==1):
                    plot.plot(percent, pft_acc)
                    plot.title('Partial Fit Accuracy')
                    plot.xlabel('Percent of Data')
                    plot.ylabel('Accuracy')
                    plot.show()

                with open(pft_path,'a', newline='') as pf:
                    writer = csv.writer(pf)
                    writer.writerow(pft_acc)

                ################### WARM START TESTING ############################
                get_cm_one_run = 0
                ws_model = deepcopy(model)
                percent, ws_acc = PFvsWS('WS',ws_model, X_test, y_test, type_model, pca, scaling, get_cm_one_run)
                score = np.array(score)
                ws_acc = np.insert(ws_acc,0,score, axis=0)
                percent = np.insert(percent, 0, 0.0)
                ws_acc = np.array(ws_acc)
                ws_acc = np.round(ws_acc,3)
                # Plot percent vs accuracy
                if(printer==1):
                    plot.plot(percent, ws_acc)
                    plot.title('Warm Start Accuracy')
                    plot.xlabel('Percent of Data')
                    plot.ylabel('Accuracy')
                    plot.show()

                with open(ws_path,'a', newline='') as ws:
                    writer = csv.writer(ws)
                    writer.writerow(ws_acc)

In [ ]:
############################ CREATE BIG CONFUSION MATRIX ############################
create_big_csv = 0
create_big_confusion_matrix = 0
if(create_big_csv == 0):
    files = ['LDA_CM.csv','SVML_CM.csv','SVMR_CM.csv',
             'MLP1_CM.csv','MLP2_CM.csv','MLP3_CM.csv',
             'NN1_CM.csv','NN2_CM.csv','NN3_CM.csv']
    names = ['LDA','SVML','SVMR','MLP1','MLP2','MLP3','NN1','NN2','NN3']

    for i in range(len(files)):
        file = r'LOOP_CM\\' + names[i] + '\\' +files[i]  
        with open(file, 'r') as f:
            df = pd.read_csv(f)
            y_true = np.array(df.iloc[:,0])
            y_pred = np.array(df.iloc[:,1]) 
            
        with open(r'LOOP_CM\\LOOP_CM.csv','a', newline='') as loop_cm:
            writer = csv.writer(loop_cm)
            for idx in range(len(y_true)):
                writer.writerow([y_true[idx],y_pred[idx]])
            
        title = names[i] + ' LOOP Confusion Matrix'
        plot_confusion_matrix(y_true, 
                            y_pred, 
                            classes=np.array(['1','2','3','4','5','6']), 
                            normalize=True, 
                            title=title)
        path = r'LOOP_CM\\'
        path = path + names[i] + '_Confusion_Matrix.png'
        plot.savefig(path)
        plot.close()
        
if(create_big_confusion_matrix == 1):
    files = 'LOOP_CM.csv'
    path = 'LOOP_CM\\' + files
    with open(path,'r') as f:
        df = pd.read_csv(f)
        y_true = np.array(df.iloc[:,0])
        y_pred = np.array(df.iloc[:,1]) 
        
    title = ' LOOP Confusion Matrix Normalized'
    plot_confusion_matrix(y_true, 
                        y_pred, 
                        classes=np.array(['1','2','3','4','5','6']), 
                        normalize=True, 
                        title=title)
    path = r'LOOP_CM\\'
    path = path + 'Confusion_Matrix_Normalized.png'
    plot.savefig(path)
    plot.close()
    
    files = 'LOOP_CM.csv'
    path = 'LOOP_CM\\' + files
    with open(path,'r') as f:
        df = pd.read_csv(f)
        y_true = np.array(df.iloc[:,0])
        y_pred = np.array(df.iloc[:,1]) 
        
    title = ' LOOP Confusion Matrix'
    plot_confusion_matrix(y_true, 
                        y_pred, 
                        classes=np.array(['1','2','3','4','5','6']), 
                        normalize=False, 
                        title=title)
    path = r'LOOP_CM\\'
    path = path + 'Confusion_Matrix.png'
    plot.savefig(path)
    plot.close()

In [ ]:
############################ GET RESUME TABLE ############################
get_mean_and_std = 0
if(get_mean_and_std == 1):
    files = ['LDA_LOOP_results.csv','SVML_LOOP_results.csv','SVMR_LOOP_results.csv'
            ,'MLP1_LOOP_results.csv','MLP2_LOOP_results.csv','MLP3_LOOP_results.csv'
            ,'NN1_LOOP_results.csv','NN2_LOOP_results.csv','NN3_LOOP_results.csv'
            ]
    names = ['LDA','SVML','SVMR'
            ,'MLP1','MLP2','MLP3'
            ,'NNET1','NNET2','NNET3'
            ]

    for i in range(len(files)):
        df = pd.read_csv(r'LOOP\\'+files[i])
        mean = np.round(np.array(np.mean(df, axis=0)),3)
        with open(r'LOOP\\'+files[i], 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(mean)
        with open(r'LOOP_All_Table\\Table_2.csv','a',newline='') as f:
            writer = csv.writer(f)
            writer.writerow(mean)
            
    files = ['LDA_LOOP_Class_results.csv','SVML_LOOP_Class_results.csv','SVMR_LOOP_Class_results.csv'
            ,'MLP1_LOOP_Class_results.csv','MLP2_LOOP_Class_results.csv','MLP3_LOOP_Class_results.csv'
            ,'NN1_LOOP_Class_results.csv','NN2_LOOP_Class_results.csv','NN3_LOOP_Class_results.csv'
            ]
    names = ['LDA','SVML','SVMR'
            ,'MLP1','MLP2','MLP3'
            ,'NNET1','NNET2','NNET3'
            ]

    for i in range(len(files)):
        df = pd.read_csv(r'LOOP\\'+files[i])
        mean = np.round(np.array(np.mean(df, axis=0)),3)
        std = np.round(np.array(np.std(df, axis=0)),3)
        with open(r'LOOP\\'+files[i], 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(mean)
            writer.writerow(std)
        with open(r'LOOP_All_class_Table\\Table_Mean_4.csv','a',newline='') as f:
            writer = csv.writer(f)
            writer.writerow(mean)
        with open(r'LOOP_All_class_Table\\Table_Std_4.csv','a',newline='') as f:
            writer = csv.writer(f)
            writer.writerow(std)

In [ ]:
read_file = 0
if read_file == 1:
    files = ['PFT\\MLP1_PartialFit.csv','WS\\MLP1_WarmStart.csv',
             'PFT\\MLP2_PartialFit.csv','WS\\MLP2_WarmStart.csv',
             'PFT\\MLP3_PartialFit.csv','WS\\MLP3_WarmStart.csv']
    for i in range(len(files)):
        df = pd.read_csv(files[i])
        print('File:',files[i])
        # get mean per column
        m = df.mean(axis=0)
        m = np.round(m,3)
        m = np.array(m)
        print(m)
        
        with open(files[i],'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(m)

In [ ]:
file = r'LOOP_All_Table\\Table_2.csv'
names = ['LDA','SVML','SVMR','MLP1','MLP2','MLP3','NNET1','NNET2','NNET3']

df = pd.read_csv(file)
mean = np.array(df.iloc[:,0])
print(len(mean))
std = np.array(df.iloc[:,1])
print(len(std))

fig, ax = plot.subplots()
ax.bar(names,mean,
       yerr=std,
       align='center',
       alpha=0.5,
       ecolor='black',
       capsize=10)
plot.ylim(40,100)
plot.xlabel('Classifier')
plot.ylabel('Validation Accuracy')
plot.title('Leave One Out Participant Validation Accuracy')
plot.savefig(r'LOOP_All_Table\\Table_2_1.png')

In [ ]:
file = r'LOOP_All_Table\\Table_2.csv'
names = ['LDA','SVML','SVMR','MLP1','MLP2','MLP3','NNET1','NNET2','NNET3']

df = pd.read_csv(file)
val_acc = np.array(df.iloc[:,0])
test_acc = np.array(df.iloc[:,-1])

plot.figure()
plot.plot(names,val_acc,label='Validation Accuracy')
plot.plot(names,test_acc,label='Test Accuracy')
plot.legend()
plot.xlabel('Classifier')
plot.ylabel('Validation Accuracy')
plot.ylim(50,100)
plot.title('Leave One Out Participant Validation Accuracy')
plot.savefig(r'LOOP_All_Table\\Table_2_2.png')

In [ ]:
################### GET MEAN PER PERCENTAGE ############################
read_file = 1
if read_file == 1:
    percent = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    files = ['PFT\\MLP1_PartialFit.csv','WS\\MLP1_WarmStart.csv',
             'PFT\\MLP2_PartialFit.csv','WS\\MLP2_WarmStart.csv',
             'PFT\\MLP3_PartialFit.csv','WS\\MLP3_WarmStart.csv']
    for i in range(len(files)):
        file = files[i]
        df = pd.read_csv(file)
        #print('File:',file)
        fx = df.iloc[-1,:]
        fx = np.array(fx)
        file = file[:-4]
        #print(file)
        
        if file[0:3] == 'PFT':
            title = file[4:8] + ' Partial Fit Accuracy'
            path = 'PFT\\'
        if file[0:2] == 'WS':
            title = file[3:7] + ' Warm Start Accuracy'
            path = 'WS\\'
            
    
        plot.figure()
        plot.plot(percent, fx)
        plot.title(title)
        plot.xlabel('Percentage of Data Used for Training')
        plot.ylabel('Testing Accuracy')
        plot.savefig(path + title + '.png')
        plot.show()    

In [ ]:
################### PLOT IN ONE FIGURE ############################
read_file = 1
if read_file == 1:
    percent = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    files = ['PFT\\MLP1_PartialFit.csv','WS\\MLP1_WarmStart.csv',
             'PFT\\MLP2_PartialFit.csv','WS\\MLP2_WarmStart.csv',
             'PFT\\MLP3_PartialFit.csv','WS\\MLP3_WarmStart.csv']
    for i in range(len(files)):
        file = files[i]
        df = pd.read_csv(file)
        #print('File:',file)
        fx = df.iloc[-1,:]
        fx = np.array(fx)
        file = file[:-4]
        #print(file)
        
        if file[0:3] == 'PFT':
            title = file[4:8] + ' Partial Fit Accuracy'
            label = file[4:8] + ' PF'
        if file[0:2] == 'WS':
            title = file[3:7] + ' Warm Start Accuracy'
            label = file[3:7] + ' WS'

        plot.plot(percent, fx, label=label)
        plot.title('Partial Fit vs Warm Start Accuracy')
        plot.xlabel('Percentage of Data Used for Training')
        #plot.ylim(80,100)
        plot.ylabel('Testing Accuracy')
        plot.savefig('WS\\PFTvsWS.png')
        plot.savefig('PFT\\PFTvsWS.png')
        plot.legend()
        